In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(f'/media/beans/ssd/bespoke')

from models import EffNet
from constants import *
from imports import *
from rw_dataloader import RealWorldDataloader
from blender_dataloader import BlenderDataloader
from train_utils import run_epoch, eval_rw

torch.__version__, torch.cuda.device_count(), torch.cuda.get_device_name(torch.cuda.current_device())

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:689: FutureWarning: This class has been deprecated. Please use CoarseDropout
  warnings.warn(
/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:965: FutureWarning: This class has been deprecated. Please use ImageCompression
  warnings.warn(


('1.10.0', 2, 'NVIDIA GeForce RTX 3090')

In [2]:
import wandb
run_id = wandb.util.generate_id(); run_id

'vxc5h7qs'

In [3]:
wandb.init(id='1h7dizrn', project="carla", resume="allow")

wandb: Currently logged in as: rgilman33 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.18 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [4]:
m = EffNet(model_arch="efficientnet_b3").to(device) # 13M params, 11.6M without RNN, 
sum([torch.numel(p) for p in m.parameters()]) / 1000

13332.809

In [5]:
m.backbone

EfficientNet(
  (conv_stem): Conv2d(3, 40, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): SiLU(inplace=True)
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=40, bias=False)
        (bn1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act1): SiLU(inplace=True)
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(40, 10, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(10, 40, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(40, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): Identity()
 

In [6]:
m.load_state_dict(torch.load(f"{BESPOKE_ROOT}/models/m.torch"), strict=False)

<All keys matched successfully>

In [7]:
scaler = torch.cuda.amp.GradScaler() 
opt = torch.optim.Adam(m.parameters(), lr=1e-4)

In [8]:
BS = 32
dataloader_trn = BlenderDataloader(path_stem="trn", bs=BS)

In [9]:
%%time
#rw_dataloader = RealWorldDataloader()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.39 µs


In [10]:
#wandb.watch(m, log="all")

In [11]:
model_stem = "6.10"
START_E = 23

In [12]:
n_epochs = 100
log_wandb = True

for e in range(START_E, n_epochs):
    
    # Train
    print(f"EPOCH {e}\n")
    run_epoch(dataloader_trn, m, opt=opt, scaler=scaler, 
              train=True, backwards=True, log_wandb=log_wandb, wandb=wandb, updates_per_epoch=2650)
    
    print("\n\n Eval...")
    
    #eval_rw(rw_dataloader, m, wandb) # will log directly to wandb
        
    # Saving model checkpoint each epoch
    torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e{e}.torch")
    

EPOCH 23

{'trn_control_loss': 0.00366999, 'consistency losses/trn_steer_cost': 0.00517283, 'consistency losses/trn_te_loss': 0.02026911, 'consistency losses/trn_%_updates_w_torque_loss': 0.421875, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.90625, 'logistical/obs_consumed_per_second': 240.04724409, 'logistical/obs_generated_per_second': 115.38582677, 'logistical/data_consumption_ratio': 2.07729789, 'logistical/manual_train_pause': 0.0, 'consistency losses/trn_torque_delta_loss': 15.5172077, 'consistency losses/trn_torque_loss': 154.69097222, 'logistical/max_param': 16.25064468, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 239.7734375, 'logistical/obs_generated_per_second': 114.921875, 'logistical/data_consumption_ratio': 2.08604042, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00391349, 'consistency losses/trn_steer_cost': 0.00634719, 'consistency losses/trn_te_loss': 0.0239318, 'consistency losses/trn_%_updates_w_torque_loss': 0.421875,

{'logistical/obs_consumed_per_second': 238.4765625, 'logistical/obs_generated_per_second': 112.328125, 'logistical/data_consumption_ratio': 2.1225992, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00335531, 'consistency losses/trn_steer_cost': 0.00505084, 'consistency losses/trn_te_loss': 0.01953398, 'consistency losses/trn_%_updates_w_torque_loss': 0.28125, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.84375, 'consistency losses/trn_torque_delta_loss': 12.63022642, 'consistency losses/trn_torque_loss': 95.04644097, 'logistical/max_param': 16.24573326, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 242.1640625, 'logistical/obs_generated_per_second': 110.875, 'logistical/data_consumption_ratio': 2.18429412, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00318164, 'consistency losses/trn_steer_cost': 0.00455934, 'consistency losses/trn_te_loss': 0.01760271, 'consistency losses/trn_%_updates_w_torque_loss': 0.28125, 'consistency loss

{'logistical/obs_consumed_per_second': 237.890625, 'logistical/obs_generated_per_second': 108.2109375, 'logistical/data_consumption_ratio': 2.2027887, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00318293, 'consistency losses/trn_steer_cost': 0.00513475, 'consistency losses/trn_te_loss': 0.01959802, 'consistency losses/trn_%_updates_w_torque_loss': 0.4140625, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.8515625, 'consistency losses/trn_torque_delta_loss': 13.34475344, 'consistency losses/trn_torque_loss': 158.20371462, 'logistical/max_param': 16.24867821, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 241.234375, 'logistical/obs_generated_per_second': 110.90625, 'logistical/data_consumption_ratio': 2.17518583, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00322589, 'consistency losses/trn_steer_cost': 0.00452225, 'consistency losses/trn_te_loss': 0.01744262, 'consistency losses/trn_%_updates_w_torque_loss': 0.2890625, 'consiste

{'trn_control_loss': 0.00329322, 'consistency losses/trn_steer_cost': 0.00537787, 'consistency losses/trn_te_loss': 0.02054154, 'consistency losses/trn_%_updates_w_torque_loss': 0.28125, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.8515625, 'consistency losses/trn_torque_delta_loss': 13.29094395, 'logistical/obs_consumed_per_second': 241.57480315, 'logistical/obs_generated_per_second': 110.07874016, 'logistical/data_consumption_ratio': 2.19934298, 'logistical/manual_train_pause': 0.0, 'consistency losses/trn_torque_loss': 110.41970486, 'logistical/max_param': 16.24678993, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 244.40625, 'logistical/obs_generated_per_second': 117.359375, 'logistical/data_consumption_ratio': 2.08243215, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00332133, 'consistency losses/trn_steer_cost': 0.00504123, 'consistency losses/trn_te_loss': 0.01923615, 'consistency losses/trn_%_updates_w_torque_loss': 0.2578125, 'consis

{'logistical/obs_consumed_per_second': 241.296875, 'logistical/obs_generated_per_second': 112.21875, 'logistical/data_consumption_ratio': 2.15484887, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00334153, 'consistency losses/trn_steer_cost': 0.00480451, 'consistency losses/trn_te_loss': 0.01812246, 'consistency losses/trn_%_updates_w_torque_loss': 0.4375, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.8203125, 'consistency losses/trn_torque_loss': 140.67215402, 'consistency losses/trn_torque_delta_loss': 15.45440848, 'logistical/max_param': 16.25280571, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 241.65625, 'logistical/obs_generated_per_second': 104.203125, 'logistical/data_consumption_ratio': 2.33084552, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00333236, 'consistency losses/trn_steer_cost': 0.00502115, 'consistency losses/trn_te_loss': 0.01917058, 'consistency losses/trn_%_updates_w_torque_loss': 0.375, 'consistency loss

...

{'trn_control_loss': 0.00284812, 'consistency losses/trn_steer_cost': 0.00460905, 'consistency losses/trn_te_loss': 0.01770537, 'consistency losses/trn_%_updates_w_torque_loss': 0.3984375, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.8203125, 'consistency losses/trn_torque_loss': 199.67340686, 'consistency losses/trn_torque_delta_loss': 13.32713914, 'logistical/obs_consumed_per_second': 238.99212598, 'logistical/obs_generated_per_second': 121.18897638, 'logistical/data_consumption_ratio': 1.97329727, 'logistical/manual_train_pause': 0.0, 'logistical/max_param': 16.24437714, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 241.4765625, 'logistical/obs_generated_per_second': 116.421875, 'logistical/data_consumption_ratio': 2.07483971, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00263957, 'consistency losses/trn_steer_cost': 0.00432865, 'consistency losses/trn_te_loss': 0.01641618, 'consistency losses/trn_%_updates_w_torque_loss': 0.2890625, 'co

{'logistical/obs_consumed_per_second': 242.296875, 'logistical/obs_generated_per_second': 114.125, 'logistical/data_consumption_ratio': 2.12942885, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00315173, 'consistency losses/trn_steer_cost': 0.0058867, 'consistency losses/trn_te_loss': 0.0222314, 'consistency losses/trn_%_updates_w_torque_loss': 0.3515625, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.890625, 'consistency losses/trn_torque_delta_loss': 14.88603344, 'consistency losses/trn_torque_loss': 152.38958333, 'logistical/max_param': 16.24730301, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 243.125, 'logistical/obs_generated_per_second': 118.484375, 'logistical/data_consumption_ratio': 2.05823688, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00298317, 'consistency losses/trn_steer_cost': 0.0052619, 'consistency losses/trn_te_loss': 0.01995842, 'consistency losses/trn_%_updates_w_torque_loss': 0.453125, 'consistency losses

{'logistical/obs_consumed_per_second': 239.4609375, 'logistical/obs_generated_per_second': 110.2578125, 'logistical/data_consumption_ratio': 2.17170138, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00292277, 'consistency losses/trn_steer_cost': 0.00524709, 'consistency losses/trn_te_loss': 0.01952788, 'consistency losses/trn_%_updates_w_torque_loss': 0.421875, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.796875, 'consistency losses/trn_torque_delta_loss': 14.05551088, 'consistency losses/trn_torque_loss': 166.99797454, 'logistical/max_param': 16.2450695, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 240.203125, 'logistical/obs_generated_per_second': 112.828125, 'logistical/data_consumption_ratio': 2.12835852, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00297071, 'consistency losses/trn_steer_cost': 0.00500412, 'consistency losses/trn_te_loss': 0.01894521, 'consistency losses/trn_%_updates_w_torque_loss': 0.3515625, 'consiste

{'trn_control_loss': 0.00264743, 'consistency losses/trn_steer_cost': 0.00478616, 'consistency losses/trn_te_loss': 0.01786001, 'consistency losses/trn_%_updates_w_torque_loss': 0.328125, 'consistency losses/trn_%_updates_w_torque_delta_loss': 0.8125, 'consistency losses/trn_torque_delta_loss': 11.32960862, 'logistical/obs_consumed_per_second': 242.82677165, 'logistical/obs_generated_per_second': 108.51968504, 'logistical/data_consumption_ratio': 2.24213259, 'logistical/manual_train_pause': 0.0, 'consistency losses/trn_torque_loss': 151.85900298, 'logistical/max_param': 16.24562645, 'logistical/lr': 0.0001}
{'logistical/obs_consumed_per_second': 238.65625, 'logistical/obs_generated_per_second': 107.4921875, 'logistical/data_consumption_ratio': 2.22334917, 'logistical/manual_train_pause': 0.0, 'trn_control_loss': 0.00279209, 'consistency losses/trn_steer_cost': 0.00522889, 'consistency losses/trn_te_loss': 0.01999742, 'consistency losses/trn_%_updates_w_torque_loss': 0.4609375, 'consist

KeyboardInterrupt: 

In [10]:
%%time

eval_rw(rw_dataloader, m, wandb) # will log directly to wandb

/home/beans/miniconda3/envs/py38/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


CPU times: user 14.3 s, sys: 417 ms, total: 14.7 s
Wall time: 14 s


In [13]:
torch.save(m.state_dict(), f"{BESPOKE_ROOT}/models/m_{model_stem}_e57.torch")